In [1]:
import tensorflow as tf

F:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
#HELPER

#INIT WEIGHTS

def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [5]:
#INIT BIAS

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [6]:
#COV2D

def conv2d(x,W):
    #x --> [batch, H, W, channels]
    #W --> [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

In [7]:
#POOLING
def max_pool_2by2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [8]:
#CONVOLUTIONAL LAYER
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [9]:
#NORMAL (FULLY CONNECTED LAYER)
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [10]:
#PLACEHOLDERS
x = tf.placeholder(tf.float32,shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [11]:
#LAYERS
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [12]:
convo_1 = convolutional_layer(x_image, shape= [5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [13]:
convo_2 = convolutional_layer(convo_1_pooling, shape= [5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [14]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [15]:
#DROPOUT
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [16]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [17]:
#LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [18]:
#OPTIMIZER
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [19]:
init = tf.global_variables_initializer()

In [20]:
steps = 1000
with tf.Session() as sess:
    sess.run(init)
    for i  in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict = {x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            print("On Step {}".format(i))
            print("Accuracy:")
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc, feed_dict={x:mnist.test.images, y_true: mnist.test.labels, hold_prob:1.0}))
            print("\n")

On Step 0
Accuracy:
0.1734


On Step 100
Accuracy:
0.9419


On Step 200
Accuracy:
0.9519


On Step 300
Accuracy:
0.97


On Step 400
Accuracy:
0.9708


On Step 500
Accuracy:
0.9744


On Step 600
Accuracy:
0.9792


On Step 700
Accuracy:
0.9826


On Step 800
Accuracy:
0.9837




ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "F:\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 959, in __init__
    self.this.append(this)
  File "F:\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 945, in <lambda>
    __getattr__ = lambda self, name: _swig_getattr(self, TF_Output, name)
  File "F:\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 74, in _swig_getattr
    return _swig_getattr_nondynamic(self, class_type, name, 0)
  File "F:\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 69, in _swig_getattr_nondynamic
    return object.__getattr__(self, name)
AttributeError: type object 'object' has no attribute '__getattr__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.us

TypeError: must be str, not list